In [1]:
# libararies
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import copy as copy
import os


MATCH KEYPOINTS FUNCTION

In [2]:
def matchKeyPoints(keyPoints1, keyPoints2, descriptors1, descriptors2):
    
    matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
    
    descriptors1 = np.float32(descriptors1)
    descriptors2 = np.float32(descriptors2)
    
    raw_matches = matcher.knnMatch(descriptors1, descriptors2, k=2)
    
    matches = []
    for m in raw_matches:
        if len(m) == 2 and m[0].distance < m[1].distance * 0.75:
            matches.append((m[0].trainIdx, m[0].queryIdx))
    
    if len(matches) > 4:
        
        keyPoints1 = np.float32([keyPoints1[i] for (_, i) in matches])
        keyPoints2 = np.float32([keyPoints2[i] for (i, _) in matches])
        
        H, status = cv.findHomography(keyPoints1, keyPoints2, cv.RANSAC, 4.0)
        
        print('%d/%d inliers/matched' % (np.sum(status), len(status)))
            
    else:
        H, status = None, None
        print('%d matches found, not enough for homography estimation' % len(matches))
    
    return matches, H, status

WIDTH DIMENSION FINDING FUNCTION

In [3]:
def findImgWidthDim(Img):
    min_idx = 10000
    max_idx = 0

    
    
    _, width, _ = Img.shape

    for idx in range(width):
        if Img[150, idx, 0] == 0 and Img[150, idx, 1] == 0 and Img[150, idx, 2] == 0:
            min_idx = idx
        else:
            break
        
    for idx in range(width):
        if Img[150, width - idx - 1, 0] == 0 and Img[150, width - idx - 1, 1] == 0 and Img[150, width - idx - 1, 2] == 0:
            max_idx = width - idx - 1
        else:
            break
        
    return min_idx, max_idx

1. IMPORT AND VISUALIZE IMAGES

In [4]:
# recall color and gray images and mask
color_ref = cv.imread('./PlanarImages/reference/Ref.jpg')
gray_ref = cv.cvtColor(color_ref, cv.COLOR_BGR2GRAY)

path_colors = './PlanarImages/image/'
path_masks = './PlanarImages/mask/'
colors = []
grays = []
masks = []

for root, directions, files in os.walk(path_colors):
    
    for file in files:
        if '.jpg' in file:
            
            img_input = cv.imread(os.path.join(root, file))
            gray_input = cv.cvtColor(img_input, cv.COLOR_BGR2GRAY)
            
            colors.append(img_input)
            grays.append(gray_input)

for root, directions, files in os.walk(path_masks):
    
    for file in files:
        if '.png' in file:
            
            mask = cv.imread(os.path.join(root, file))
            masks.append(mask)
            

2. EXTRACT FEATURES

In [5]:
# feature detector
detector = cv.BRISK_create()
key_pts_ref, descriptors_ref = detector.detectAndCompute(gray_ref, None)
key_pts_ref = np.float32([key_pt.pt for key_pt in key_pts_ref])

    # referenece image with buffer
buffer = 300
height_ref, width_ref, channel_ref = color_ref.shape
color_ref_buffer = np.zeros((height_ref + 2*buffer, width_ref + 2*buffer, channel_ref), dtype=np.uint8)
color_ref_buffer[0:height_ref, buffer-1:width_ref + buffer-1] = color_ref

gray_ref_buffer = cv.cvtColor(color_ref_buffer, cv.COLOR_BGR2GRAY)
key_pts_ref_buffer, descriptors_ref_buffer = detector.detectAndCompute(gray_ref_buffer, None)
key_pts_ref_buffer = np.float32([key_pt.pt for key_pt in key_pts_ref_buffer])


key_pts = []
descriptors = []

for i in range(4):
    
    key_pt, descriptor = detector.detectAndCompute(grays[i], None)
    key_pt = np.float32([k_pt.pt for k_pt in key_pt])
    key_pts.append(key_pt)
    descriptors.append(descriptor)


3. MATCHING w/ RANSAC

In [11]:
height_ref_buffer, width_ref_buffer, _ = color_ref_buffer.shape

img_combined = color_ref_buffer.copy()
min_idx, max_idx = findImgWidthDim(img_combined)
print('min: %d, max: %d' % (min_idx, max_idx))
for i in range(4):
    key_pts_image = key_pts[i].copy()
    descriptors_image = descriptors[i].copy()
    matches, H, status = matchKeyPoints(key_pts_ref_buffer, key_pts_image, descriptors_ref_buffer, descriptors_image)
    H_inv = np.linalg.inv(H)
    H_inv = H_inv / H_inv[2,2]
    result = cv.warpPerspective(colors[i], H_inv, 
                                (width_ref_buffer, height_ref_buffer))
    result[0:img_combined.shape[0], min_idx:max_idx] = img_combined[0:img_combined.shape[0], min_idx:max_idx]
    img_combined = result.copy()
    min_idx, max_idx = findImgWidthDim(img_combined)



cv.imshow('result', result)
cv.waitKey(10000)
cv.destroyAllWindows()


min: 298, max: 1208
4668/4841 inliers/matched
2571/2727 inliers/matched
2369/2709 inliers/matched
2685/2839 inliers/matched
